In [ ]:
import coverage
coverage.process_startup()

## 1. Prerequisites for Using dpest

### 1.1. Using Pre-generated DSSAT Files

*dpest* is designed to interface with the Decision Support System for Agrotechnology Transfer (DSSAT), a Windows-based software application program, for crop simulation. To enable verification without DSSAT installation, this repository includes pre-generated DSSAT files in a standardized directory structure, from the **SWSW7501WH N RESPONSE experiment** included in the **CERES-Wheat model**:

```
DSSAT48/  
├── Genotype/          # Contains cultivar and ecotype parameter files  
│   ├── WHCER048.CUL  
│   └── WHCER048.ECO
└── Wheat/            # Contains experiment input and output files
    ├── SWSW7501.WHT
    ├── OVERVIEW.OUT
    └── PlantGro.OUT
```

These files mimic the structure of a DSSAT installation (`C:/DSSAT48/`) but are self-contained within this repository. Examples reference these files using **relative paths**, ensuring portability.

**Full use of dpest requires DSSAT (v4.8+), freely available at the [DSSAT website](https://dssat.net/).** 

### 1.2. How the Provided DSSAT Files Were Generated

If you wish to reproduce the DSSAT files yourself, follow these steps:

1. Launch DSSAT.
2. Click “Selector”.
3. Expand “Crops” and select “Wheat”.
4. In the “Data” panel select the “SWSW7501.WHX” experiment.
5. Click “Run” button in the toolbar.
6. In the “Simulation” popup window, choose “CERES” as the crop model.
7. Click “Run Model” and wait for the simulation to finish.


[![](https://markdown-videos-api.jorgenkh.no/youtube/dzKpvJSEXZc?vq)](https://www.youtube.com/embed/dzKpvJSEXZc?vq=hd1080)

Click to watch the steps!

### 1.2 Load the *dpest* library or import each module

In [ ]:
#from dpestool.wheat.ceres import cul, eco
# from dpestool.wheat import overview, plantgro
# from dpestool import pst
# from dpestool.wheat.utils import uplantgro

import dpest 

## 2. Instructions for Creating the PEST Input Files

This section demonstrates how to use the `dpest` library functions to create the **PEST input files** with practical examples. For each function, you will find:

- Example code showing how to call the function and the expected output messages.
- Output files generated by running these examples. These files will appear in your **current working directory**.

These examples can be used as templates for your workflows or as a reference to verify your installation and usage of `dpest`.

**Important:**  
All files generated by `dpest` (such as `PEST_CONTROL.pst`, `.TPL`, `.INS`, etc.) will appear in your current working directory. This is necessary because the PEST utility `pestchek.exe` expects both the control file and all referenced files to be in the directory where the command is run, and does not accept relative paths to files located in subdirectories. Saving all files in the current directory ensures that validation and further PEST operations work correctly.

After running the examples, you should see the following files and folders in your working directory:

```
basic_usage.ipynb
OVERVIEW.ins
PEST_CONTROL.pst
PlantGro.ins
WHCER048_CUL.TPL
WHCER048_ECO.TPL
```

You will learn how to validate the generated PEST files using `pestchek.exe` in section `4. Validate the Created PEST Input Files`.

---

### 2.1. Creating CULTIVAR Parameters TPL File

The first step is to create the cultivar Template File (`.TPL`) for the `MANITOU` cultivar, which is the cultivar planted in the `164.0 KG N/HA IRRIG` treatment of the `SWSW7501.WHX` experiment. To achieve this, we use the `dpest.wheat.ceres.cul()` and the DSSAT `WHCER048.CUL` file.

The returned `cultivar_parameters` dictionary will be used later to make the control file’s parameter groups and parameters sections using the `pst` module. The `cultivar_tpl_path` path will be used in the `input_output_file_pairs` argument of the pst module to match the original cultivar file to the PEST template file (`.TPL`).

In [ ]:
# Call the function 
cultivar_parameters, cultivar_tpl_path = dpest.wheat.ceres.cul(
    P = 'P1D, P5', 
    G = 'G1, G2, G3', 
    PHINT = 'PHINT',
    cultivar = 'MANITOU',
    cul_file_path = './DSSAT48/Genotype/WHCER048.CUL'
)

# Print The returned tuple
print('\nCultivar parameters:', cultivar_parameters)

### 2.2. Creating ECOTYPE Parameters TPL File

Usually, the DSSAT models do not require updating the Cultivar parameters. However, in this example we are going to update some of the Cultivar parameters based on the DSSAT `WHCER048.ECO file`, using the `dpest.wheat.ceres.eco()` function.

The returned `ecotype_parameters` dictionary will be used later to make the control file’s parameter groups and parameters sections using the `pst` module. The `ecotype_tpl_path` path will be used in the `input_output_file_pairs` argument of the pst module to match the original cultivar file to the PEST template file (`.TPL`).

In [ ]:
# Call the function 
ecotype_parameters, ecotype_tpl_path = dpest.wheat.ceres.eco(
    PHEN = 'P1, P2FR1',
    VERN = 'VEFF',
    ecotype = 'CAWH01',
    eco_file_path = './DSSAT48/Genotype/WHCER048.ECO'
)

# Print The returned tuple
print('\nEcotype_parameters:', ecotype_parameters)

### 2.3. Creating OVERVIEW Observations INS File

Creates a `PEST instruction file (.INS)`. This instruction file contains directions for PEST to read the simulated values from the `OVERVIEW.OUT` file and compare them with the corresponding observed values (originally entered in the DSSAT “A file”). The `PEST instruction file (.INS)` guides PEST in extracting specific model-generated observations from the `OVERVIEW.OUT` file, which includes a list of end-of-season crop performance metrics, and critical phenological observations used for model evaluation. Additionally, this module creates a tuple containing:

1) A DataFrame with the MEASURED observations from the `OVERVIEW.OUT` file (originally entered in the DSSAT “A file”).
2) The path to the generated `PEST instruction file (.INS)`.

In [ ]:
# Call the function
overview_observations,  overview_ins_path = dpest.wheat.overview(
    treatment = '164.0 KG N/HA IRRIG',
    overview_file_path = './DSSAT48/Wheat/OVERVIEW.OUT'
)

# Print The returned dataframe
print('\nOverview observations:', overview_observations)

# Open the file and display the first 10 lines
with open(overview_ins_path, 'r') as file:
    lines = file.readlines()
    print(f"\nFirst 10 lines:\n{''.join(lines[:10]).strip()}")

### 2.4. Creating PlantGro Observations INS File

Creates a ``PEST instruction file (.INS)``. This instruction file contains directions for PEST to read the simulated time-series values from the ``PlantGro.OUT`` file, which includes various plant growth metrics throughout the growing season. The  ``PEST instruction file (.INS)`` guides PEST in extracting specific model-generated observations for specific time points from the ``PlantGro.OUT`` file for the variables specified by the user. Additionally, this module creates a tuple containing: 

1) A DataFrame with the MEASURED observations (entered by the user in the DSSAT "T file") for the specified variables, and 
2) the path to the generated ``PEST instruction file (.INS)``.

In [ ]:
# Call the function
plantgro_observations, plantgro_ins_path = dpest.wheat.plantgro(
    plantgro_file_path = './DSSAT48/Wheat/PlantGro.OUT',
    treatment = '164.0 KG N/HA IRRIG',
    variables = ['LAID', 'CWAD', 'T#AD']
)

# Print The returned dataframe
print('\nPlantgro observations:\n', plantgro_observations)

# Open the file created and display the first 10 lines
with open(plantgro_ins_path, 'r') as file:
    lines = file.readlines()
    print(f"\nFirst 10 lines:\n{''.join(lines[:10]).strip()}")

### 2.5. Creating the PST File

 Creates the ``PEST control file (.PST)`` for DSSAT CERES wheat crop model calibration. This file guides the model calibration process by specifying input and output files, parameter bounds, and directions for PEST to extract and compare model-generated observations with experimental data. The module takes model parameters (with their values, groupings, and bounds) and observation DataFrames as inputs.

The ``.PST`` file is created using the ``variables`` obtained in ``2.1``, ``2.2``, ``2.3``, and ``2.4``. Additionally, we need to specify the ``command-line instruction`` to execute the DSSAT model. For more on running DSSAT from the command line and managing outputs, see the **[DSSAT Power Users Guide](https://dssat.net/tools/tools-for-power-users/).**.

The following Python script provides an example of how to run the ``DSSAT CERES-Wheat model`` using Python. 

```Python
import os
import subprocess
from dpest.wheat.utils import uplantgro

# User-editable section for system DSSAT installation
dssat_install_dir = r'C:\DSSAT48'  # System DSSAT installation folder
dssat_exe = os.path.join(dssat_install_dir, 'DSCSM048.EXE')
control_file = os.path.join(dssat_install_dir, 'Wheat', 'DSSBatch.v48')

# Project data directory (relative to script location)
project_dir = os.path.dirname(os.path.abspath(__file__))
data_dir = os.path.join(project_dir, 'DSSAT48')
output_dir = os.path.join(data_dir, 'Wheat')

# Change working directory to the output directory
os.chdir(output_dir)

# Build and run DSSAT command
module = 'CSCER048'
switch = 'B'
command_line = f'"{dssat_exe}" {module} {switch} "{control_file}"'
result = subprocess.run(command_line, shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

# Use uplantgro from dpest.wheat.utils to extract and update data from PlantGro.OUT if needed
uplantgro(
    plantgro_file_path=os.path.join(output_dir, 'PlantGro.OUT'),
    treatment='164.0 KG N/HA IRRIG',
    variables=['LAID', 'CWAD', 'T#AD']
)


````

**Download the example of a Python script to run DSSAT:**  

**[run_dssat.py](./run_dssat.py)** *(Click to download if not already in your directory)*

> **Important:**
> 
> The provided run_dssat.py script is set up so that DSSAT writes its output files directly into the project’s data directory (e.g., DSSAT48/Wheat). This ensures PEST always reads the latest simulation results.
>
> If you use a different method to run DSSAT (such as your own script, a batch file, or a direct executable call), you must:
>
> Ensure that DSSAT outputs are written to the correct directory referenced in your .pst file.
> Update the `* model command line` in the `.pst` file to match your actual execution command.
> Double-check that the output files are being updated with each run, so PEST uses the latest results.
>
> The run_dssat.py script is provided as a reference. Adapt it as needed for your own DSSAT installation and workflow.


**Where to save and how to call the Python script for PEST**

The Python script `run_dssat.py` is configured to be saved in the root directory of your project (i.e., in the same folder as your main project files and the `DSSAT48` data directory).

When specifying the command to execute this script in the PEST control file (`.PST`), use a command that correctly references the script’s filename and its path relative to the directory where you run PEST.

For example, if the script is named `run_dssat.py` and is located in the project root, the command to execute it would be:

```
py ./run_dssat.py
```
or equivalently:
```
python ./run_dssat.py
```

This command should be included exactly as shown in the `* model command line` section of your `.PST` file.

**Generate the PEST Control File (.PST)**  

Once the script is saved, we can generate the ``PEST control file`` using the following function:


In [ ]:
# Call the function
dpest.pst(
    cultivar_parameters = cultivar_parameters,
    ecotype_parameters = ecotype_parameters,
    dataframe_observations = [overview_observations, plantgro_observations],
    model_comand_line = r'py ./run_dssat.py',
    input_output_file_pairs = [
        ('WHCER048_CUL.TPL', './DSSAT48/Genotype/WHCER048.CUL'),
        ('WHCER048_ECO.TPL', './DSSAT48/Genotype/WHCER048.ECO'),
        ('OVERVIEW.ins', './DSSAT48/Wheat/OVERVIEW.OUT'),
        ('PlantGro.ins', './DSSAT48/Wheat/PlantGro.OUT')
    ]
)

# Open the created file and display the content
with open('PEST_CONTROL.pst', 'r') as file:
    content = file.read()  
    print('\n\n', content)  

### 2.6. Using the *uplantgro()* Function

Extends the ``PlantGro.OUT`` file by adding rows to ensure that simulated values exist for all measured observation dates. This situation arises during the calibration process when PEST attempts to compare a measured value from the DSSAT "T file" to a corresponding simulated value in the ``PlantGro.OUT`` file. If the simulation ends *before* the date of a measured observation, PEST will terminate the calibration process due to a missing observation error. This often occurs when measurements, such as remote sensing data, are taken close to the plant's maturity phase.

- This module addresses this issue by adding rows to the ``PlantGro.OUT`` file with default values (0), extending the simulation period to cover all measured observation dates.


In [ ]:
# Call the function
dpest.wheat.utils.uplantgro(
    './DSSAT48/Wheat/PlantGro.OUT', 
    '164.0 KG N/HA IRRIG', 
    ['LAID', 'CWAD', 'T#AD'])

### 3. Update the PEST_CONTROL.pst File  

This section shows how to use `dpest.utils` functions to programmatically edit a PEST control file. These utilities automate common tasks like updating optimization parameters or removing unnecessary sections, ensuring your `.pst` file is correctly configured for your workflow.  

**Why this matters**:  
The PEST control file defines how calibration runs, including solver settings, parameter groups, and observation data. Manually editing it is error-prone; these tools ensure consistency and save time.  

*For details on PEST control file structure, review the [official documentation](https://pesthomepage.org/documentation).*  

### 3.1. Updates the NOPTMAX Parameter in the PEST Control (.pst) File.

Updates the `NOPTMAX` parameter in a `PEST control (.pst)` file.

The `NOPTMAX` parameter specifies the number of optimization iterations or model runs that PEST will perform.
This function allows you to set `NOPTMAX` to any integer value, such as 0 (run the model once, no optimization) or a large value for iterative optimization.


In [ ]:
# Open the file created and display the first 10 lines
with open('PEST_CONTROL.pst', 'r') as file:
    lines = file.readlines()
    print(f"\nOriginal file. First 10 lines:\n{''.join(lines[:10]).strip()}")

# Call the function
from dpest.utils import noptmax

# Update the noptmax value
noptmax('PEST_CONTROL.pst', new_value = 50)

# Open the file created and display the first 10 lines
with open('PEST_CONTROL.pst', 'r') as file:
    lines = file.readlines()
    print(f"\nModified file. First 10 lines:\n{''.join(lines[:10]).strip()}")

### 3.2. Remove the Split Columns From the `parameter groups section`

Removes `SPLITTHRESH`/`SPLITRELDIFF`/`SPLITACTION` columns from * parameter groups section, overwriting the original file by default.

In [ ]:
# Open the file created and display the first 10 lines
with open('PEST_CONTROL.pst', 'r') as file:
    lines = file.readlines()
    print(f"\nOriginal file. Show the parameter groups section:\n{''.join(lines[14:20]).strip()}")

# Call the function
from dpest.utils import rmv_splitcols

# Delete the Split Columns
rmv_splitcols('PEST_CONTROL.pst')

# Open the file created and display the first 10 lines
with open('PEST_CONTROL.pst', 'r') as file:
    lines = file.readlines()
    print(f"\nModified file. Show the parameter groups section:\n{''.join(lines[14:20]).strip()}")

## 4.  Validate the Created PEST Input Files

The following instructions will help you validate that the PEST input files created by DPEST are correct and ready for use. To perform these validations, you need to have PEST installed on your system. You can download PEST from [https://pesthomepage.org/programs](https://pesthomepage.org/programs).

Follow each validation step as described below. After running each check, carefully review the output messages:

- If all checks report **"No errors detected"** or similar, your template files (`.TPL`), instruction files (`.INS`), and control files (`.PST`) are correctly formatted and ready for use with PEST. You can proceed confidently with your PEST workflow.


### 4.1. Validate that the files created exist in the directory 

In [ ]:
import os
import subprocess

result = subprocess.run('dir', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

### 4.2. Validate the WHCER048_CUL.TPL File

In [ ]:
result = subprocess.run('tempchek.exe WHCER048_CUL.TPL', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

### 4.3. Validate the WHCER048_ECO.TPL File

In [ ]:
result = subprocess.run('tempchek.exe WHCER048_ECO.TPL', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

### 4.4. Validate the Overview Instruction File (.INS)

In [ ]:
result = subprocess.run('inschek.exe OVERVIEW.ins ./DSSAT48/Wheat/OVERVIEW.OUT', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

### 4.5. Validate the PlantGro Instruction File (.INS)

In [ ]:
result = subprocess.run('inschek.exe PlantGro.ins ./DSSAT48/Wheat/PlantGro.OUT', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

### 4.6. Validate the PEST Control File (.PST)

In [ ]:
result = subprocess.run('pestchek.exe PEST_CONTROL.pst', shell=True, check=True, capture_output=True, text=True)
print(result.stdout)

## 5. Run the Calibration

After successfully validating the `PEST input files`, the final step is to run the calibration process.

Run the following command in the Command Prompt (or terminal, if using a different operating system) to start ``PEST`` in parameter estimation mode:

```bash
PEST.exe PEST_CONTROL.pst
```